# Nuclio - Generator function

## Environment

In [22]:
# nuclio: ignore
import nuclio

In [23]:
#%run set_env.ipynb

### Configurations

In [24]:
import os
import datetime

In [42]:
### NOTE
## Platform version dependent
base_image='iguazio/jupyter-all:2.8_b119_20200417000155'

In [43]:
# nuclio: ignore
spec = nuclio.ConfigSpec(config={"spec.triggers.inference.kind":"cron",
                                "spec.triggers.inference.attributes.interval" :"1h",
                                "spec.readinessTimeoutSeconds" : 60,
                                "spec.minReplicas" : 1,
                                "spec.build.baseImage" : base_image },
                         env={'PYTHON_SCRIPT' : '/kv-to-parquet.py',
                            'V3IO_SCRIPT_PATH' : os.path.join('/v3io','users',os.getenv('V3IO_USERNAME'),'sparkworkflows'),
                            'SPARK_JOB_NAME' : 'my-spark-job' ,
                            'SPARK_SPEC_MEM' :'2g',
                            'SPARK_SPEC_CPU' : 1,
                            'SPARK_SPEC_REPLICAS' : 1,
                              'V3IO_HOME' : os.getenv('V3IO_HOME'),
                              'V3IO_ACCESS_KEY' : os.getenv('V3IO_ACCESS_KEY'),
                              'V3IO_USERNAME' : os.getenv('V3IO_USERNAME'),
                              'V3IO_HOME_URL' : os.getenv('V3IO_HOME_URL'),
                              
                         },
                         cmd=['pip install mlrun v3io'],
                        ).with_v3io().add_volume('/User',os.path.join('/users',os.getenv('V3IO_USERNAME')),kind='v3io',name='v3io')

### Kubeflow pipeline.

In [26]:
#%run kubeflow-pipeline.ipynb

## Test

In [27]:
%%time
# nuclio: ignore
event = nuclio.Event(body='')
output = handler(context, event)
output

CPU times: user 39.9 ms, sys: 237 µs, total: 40.2 ms
Wall time: 60.1 ms


# Deploy

In [45]:
# nuclio: ignore
#%nuclio deploy -p netops -n generator -c
addr = nuclio.deploy_file(source='kubeflow-pipeline.ipynb',name='pipeline',project='sparkk',spec=spec, tag='v1.1')


[nuclio] 2020-04-20 18:27:47,635 (info) Build complete
[nuclio] 2020-04-20 18:27:56,835 (info) Function deploy complete
[nuclio] 2020-04-20 18:27:56,842 done updating pipeline, function address: 3.12.248.124:30111
